In [1]:
#task 6 python codes for longitudinal analysis

from IPython.display import JSON
from truveta.study import Client
client = Client()

import pathlib #this is a package for finding filepaths. 
import os
import pandas as pd 
import numpy as np 
from datetime import date
 
# Returns the current local date
today = date.today()

print("Today date is: ", today)

To sign in to Truveta, use a web browser to open the page https://login.truveta.com/activate?user_code=WPKH-NWGZ (User code: WPKH-NWGZ).
Signed in successfully.
Initialized Truveta Study Client (TraceId: d16ff654598b21edade65e8388442064 org: org_TfbnAclWXIW7puUv workspace: rg1)
Today date is:  2023-06-26


In [2]:
#Indicate study name: 
st = client.get_study(title="Hypertension Cohort")
assert st is not None

#Indicate population name: 
p = st.get_population(title="Study Cohort")
assert p is not None

p.get_snapshots()
#This gets the population snapshot.

population = p.get_snapshots()[0]
population.get_status()

{'id': 'ps-fxx24ulya56urj6qhcj5isknxi',
 'status': 'Finished',
 'currentStage': None,
 'totalStages': 9,
 'percentComplete': 100}

In [4]:
### This creates the data folder
data = pathlib.Path('./data')
data.mkdir(parents=True, exist_ok=True)

### This defines the load_and_dump function. 
def load_and_dump(table: str):
    print("Loading:", table)
    df = population.load_data(table=table)
    df.to_parquet(data / f"{table}.parquet", coerce_timestamps = 'us', allow_truncated_timestamps = True)

In [23]:
search_result = pd.read_parquet("data/absolute_date_time.parquet")

In [24]:
import datetime as dt
search_result['ShiftedTime'] = pd.to_datetime(search_result.AbsoluteEffectiveDateTime)- pd.DatetimeIndex(search_result.RecordedDateTime)
#take a average gap for each patient
person_shift_time = search_result.groupby('PersonId')['ShiftedTime'].mean().reset_index()

In [25]:
person_shift_time['ShiftedDays'] = person_shift_time['ShiftedTime'].dt.days
person_shift_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1206910 entries, 0 to 1206909
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype          
---  ------       --------------    -----          
 0   PersonId     1206910 non-null  object         
 1   ShiftedTime  1206910 non-null  timedelta64[ns]
 2   ShiftedDays  1206910 non-null  int64          
dtypes: int64(1), object(1), timedelta64[ns](1)
memory usage: 27.6+ MB


In [26]:
bh  = pd.read_parquet("data/body_height_vitla.parquet")
bw  = pd.read_parquet("data/body_weight_vital.parquet")

In [27]:
bh_date= bh.merge(person_shift_time, left_on="PersonId", right_on = 'PersonId')
bh_date.head()

,CodeConceptId,EffectiveDateTime,Id,EncounterId,InterpretationConceptId,NormalizedValueNumeric,NormalizedValueUOMConceptId,PatientId,PersonId,RecordedDateTime,StatusConceptId,Value,ShiftedTime,ShiftedDays
0,779602,2018-02-22 16:04:00,f3003f77-38de-616a-7754-f42ace2b3bb3,c054597c-f777-6bab-12dc-1ec9142ffecb,1067557,70.0,1190307,bcc896c2-3cbd-a1e8-1e3b-9cacce7d34fc,ce99ebb3-31b4-c108-ae51-cf4e338579c3,2018-02-22 16:04:00,1067557,70.000,497 days 12:00:00,497
1,779602,2020-12-30 09:22:00,6ec4d1ad-5f08-8af7-d690-ea1e29a18eac,0a064e31-a3ef-3e80-adae-dce8619c5f30,1067557,70.0,1190307,bcc896c2-3cbd-a1e8-1e3b-9cacce7d34fc,ce99ebb3-31b4-c108-ae51-cf4e338579c3,2020-12-30 09:22:00,1067557,70,497 days 12:00:00,497
2,779602,2020-07-29 14:39:00,9c61e163-6b89-82bb-58f5-f3ba4b8a1fc2,21fee33a-41dc-ba6e-96bd-c0532202a3bb,1067557,70.0,1190307,bcc896c2-3cbd-a1e8-1e3b-9cacce7d34fc,ce99ebb3-31b4-c108-ae51-cf4e338579c3,2020-07-29 14:39:00,1067557,70,497 days 12:00:00,497
3,779602,2021-02-24 13:12:00,875f15af-5637-2a18-8a7a-c04c90c4d8b4,c40ce441-9265-5b19-66f0-0b11aeb0001b,1067557,70.0,1190307,bcc896c2-3cbd-a1e8-1e3b-9cacce7d34fc,ce99ebb3-31b4-c108-ae51-cf4e338579c3,2021-02-24 13:12:00,1067557,70,497 days 12:00:00,497
4,779602,2018-05-18 07:39:00,55e1184b-845d-5776-b342-cf860774326c,81a78607-eea2-90f3-c796-691bf5f69e68,1067557,70.0,1190307,bcc896c2-3cbd-a1e8-1e3b-9cacce7d34fc,ce99ebb3-31b4-c108-ae51-cf4e338579c3,2018-05-18 07:39:00,1067557,70.000,497 days 12:00:00,497


In [28]:
bh_date['AbsRecordedDateTime'] =pd.to_datetime(bh_date.RecordedDateTime + bh_date.ShiftedTime)

In [33]:
bh_date.groupby(bh_date.AbsRecordedDateTime.dt.year).PersonId.nunique().reset_index()


,AbsRecordedDateTime,PersonId
0,2003.0,1
1,2004.0,2
2,2009.0,3
3,2010.0,2
4,2011.0,22
5,2012.0,15
6,2013.0,27
7,2014.0,27
8,2015.0,3663
9,2016.0,34449


In [31]:

bh_date.groupby(bh_date.AbsRecordedDateTime.dt.year).Id.nunique().reset_index()


,AbsRecordedDateTime,Id
0,2003.0,1
1,2004.0,2
2,2009.0,3
3,2010.0,2
4,2011.0,37
5,2012.0,23
6,2013.0,80
7,2014.0,88
8,2015.0,4208
9,2016.0,118367


In [36]:
bh.groupby('PersonId').Value.count().reset_index().describe()

,Value
count,104901.000000
mean,13.535257
std,16.188112
min,1.000000
25%,3.000000
50%,8.000000
75%,18.000000
max,272.000000


In [41]:
bh_value_cnt = bh.groupby('PersonId').Value.count().reset_index()
for i in range(1,5):
    print(sum(bh_value_cnt.Value >= i))

104901
89034
80867
74348


In [30]:
# distribution of body weight records and values

bw_date= bw.merge(person_shift_time, left_on="PersonId", right_on = 'PersonId')
bw_date['AbsRecordedDateTime'] =pd.to_datetime(bw_date.RecordedDateTime + bw_date.ShiftedTime)
bw_date.groupby(round(bw_date['AbsRecordedDateTime'].dt.year,0)).PersonId.nunique().reset_index()

,AbsRecordedDateTime,PersonId
0,2003.0,1
1,2004.0,2
2,2009.0,3
3,2010.0,2
4,2011.0,24
5,2012.0,18
6,2013.0,40
7,2014.0,43
8,2015.0,3719
9,2016.0,33175


In [32]:
bw_date.groupby(round(bw_date['AbsRecordedDateTime'].dt.year,0)).Id.nunique().reset_index()

,AbsRecordedDateTime,Id
0,2002.0,7
1,2003.0,14
2,2004.0,4
3,2005.0,8
4,2006.0,14
5,2007.0,19
6,2008.0,23
7,2009.0,22
8,2010.0,18
9,2011.0,108


In [37]:
bw.groupby('PersonId').Value.count().reset_index().describe()

,Value
count,106931.000000
mean,16.039558
std,19.327042
min,1.000000
25%,3.000000
50%,9.000000
75%,22.000000
max,343.000000


In [42]:
bw_value_cnt = bw.groupby('PersonId').Value.count().reset_index()
for i in range(1,5):
    print(i, ": ", sum(bw_value_cnt.Value >= i))

106931
93956
83905
77962


In [19]:
# get cleaned blood pressure data from observation table

sql = """select v.CodeConceptId, v.Id as VitalSignId, v.EffectiveDateTime, e.Id as EncounterId,
v.Value, v.PersonId, v.PatientId, v.RecordedDateTime, v.StatusConceptId,e.ClassConceptId, NormalizedValueUOMConceptId,
e.TypeConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where v.CodeConceptId = 825889 and v.CategoryConceptId = 1065654
and v.Value is not NULL
and e.ClassConceptId not in (0,1067561,2649591)
and e.ClassConceptId is not NULL
and v.NormalizedValueUOMConceptId not in (0,1067561,1067557)
and v.EffectiveDateTime is not NULL;"""

bp = population.load_sql_table(sql = sql)

In [12]:
bp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33688 entries, 0 to 33687
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   CodeConceptId                  33688 non-null  int64         
 1   EffectiveDateTime              33688 non-null  datetime64[ns]
 2   EncounterId                    33688 non-null  object        
 3   Id                             33688 non-null  object        
 4   InterpretationConceptId        33688 non-null  int64         
 5   NormalizedValueNumeric         0 non-null      object        
 6   NormalizedValueUOMConceptId    33688 non-null  int64         
 7   PatientId                      33688 non-null  object        
 8   RecordedDateTime               33688 non-null  datetime64[ns]
 9   StatusConceptId                33688 non-null  int64         
 10  Value                          33688 non-null  object        
 11  PersonId       

In [31]:
import re

In [39]:
value_list = pd.DataFrame(data = bp.Value.apply(lambda x: x.split("/")))

In [41]:
value_list['ABPValue'] = value_list.Value.apply(lambda x: x[0])
value_list['DBPValue'] = value_list.Value.apply(lambda x: x[1])
value_list.head()

,Value,ABPValue,DBPValue
0,"[130, 90]",130,90
1,"[124, 88]",124,88
2,"[152, 76]",152,76
3,"[147, 75]",147,75
4,"[102, 64]",102,64


In [11]:
sql  = """select 'Total' as Type, CodeConceptId, count(distinct Id) as nRecords, count(distinct PersonId) as nPatient
from Vital 
where CodeConceptId in (779767,779749,825889)
group by CodeConceptId
UNION
select 'Usable Enct Class' as Type, v.CodeConceptId, count(distinct v.Id) as nRecords, count(distinct v.PersonId) as nPatient
from Vital v join Encounter e on (v.EncounterId = e.Id)
where CodeConceptId in (779767,779749,825889)
and e.ClassConceptId not in (0,1067561,2649591)
and e.ClassConceptId is not NULL 
group by CodeConceptId
UNION
select 'Non-null Status' as Type, v.CodeConceptId, count(distinct v.Id) as nRecords, count(distinct v.PersonId) as nPatient
from Vital v join Encounter e on (v.EncounterId = e.Id)
where CodeConceptId in (779767,779749,825889)
and e.ClassConceptId not in (0,1067561,2649591)
and e.ClassConceptId is not NULL 
and v.StatusConceptId not in (0,1067557,1067561,2649591)
group by CodeConceptId
UNION
select 'Usable Unit' as Type, v.CodeConceptId, count(distinct v.Id) as nRecords, count(distinct v.PersonId) as nPatient
from Vital v join Encounter e on (v.EncounterId = e.Id)
where CodeConceptId in (779767,779749,825889)
and e.ClassConceptId not in (0,1067561,2649591)
and e.ClassConceptId is not NULL 
and v.StatusConceptId not in (0,1067557,1067561,2649591)
and v.NormalizedValueUOMConceptId not in (0,1067561,1067557)
group by CodeConceptId
UNION
select 'NonNull Value' as Type, v.CodeConceptId, count(distinct v.Id) as nRecords, count(distinct v.PersonId) as nPatient
from Vital v join Encounter e on (v.EncounterId = e.Id)
where CodeConceptId in (779767,779749,825889)
and e.ClassConceptId not in (0,1067561,2649591)
and e.ClassConceptId is not NULL 
and v.StatusConceptId not in (0,1067557,1067561,2649591)
and v.NormalizedValueUOMConceptId not in (0,1067561,1067557)
and v.Value is not NULL
group by CodeConceptId
order by 1,2;
"""

report = population.load_sql_table(sql = sql)
report

,Type,CodeConceptId,nRecords,nPatient
0,Non-null Status,779749,9259300,74918
1,Non-null Status,779767,7797862,64541
2,Non-null Status,825889,5378200,135750
3,NonNull Value,779749,9253793,74913
4,NonNull Value,779767,6166799,64381
5,Total,779749,10718488,131840
6,Total,779767,32599400,360491
7,Total,825889,91739025,1026691
8,Usable Enct Class,779749,10648293,128954
9,Usable Enct Class,779767,31812866,356167


In [54]:
#statistics on records and patients
bp_ptcnt = bp.groupby('ClassConceptId').PersonId.nunique().reset_index()
bp_ptcnt_type = bp_enct.merge(concept, left_on = "ClassConceptId", right_on = 'ConceptId', how = "left")[['ClassConceptId','ConceptName','PersonId']]

In [55]:
bp_enct_type

,ClassConceptId,ConceptName,PersonId
0,1065216,ambulatory,391
1,1065217,emergency,238
2,1065219,home health,24
3,1065220,inpatient encounter,211
4,1065225,observation encounter,53
5,1065227,short stay,1
6,1065230,virtual,1
7,1067555,Invalid,33


In [57]:
bp_vscnt = bp.groupby('ClassConceptId').VitalSignId.count().reset_index()
bp_vscnt_type = bp_vscnt.merge(concept, left_on = "ClassConceptId", right_on = 'ConceptId', how = "left")[['ClassConceptId','ConceptName','VitalSignId']]
bp_vscnt_type

,ClassConceptId,ConceptName,VitalSignId
0,1065216,ambulatory,17348
1,1065217,emergency,2504
2,1065219,home health,43
3,1065220,inpatient encounter,12506
4,1065225,observation encounter,1104
5,1065227,short stay,13
6,1065230,virtual,1
7,1067555,Invalid,169


In [62]:
bp_unit = bp.merge(concept,left_on = "NormalizedValueUOMConceptId", \
                   right_on = "ConceptId",how = "left").groupby( ["NormalizedValueUOMConceptId",'ConceptName']).VitalSignId.count().reset_index()

In [63]:
bp_unit

,NormalizedValueUOMConceptId,ConceptName,VitalSignId
0,1190123,millimeter mercury column,33688


In [ ]:
sql = """select v.CodeConceptId, v.Id as VitalSignId, v.EffectiveDateTime, e.id as EncounterId,
v.Value, v.PersonId, v.PatientId, v.RecordedDateTime, v.StatusConceptId,e.ClassConceptId, NormalizedVlueUOMConceptId,
e.TypeConceptId
from Vital v join Encounter e on (v.EncounterId = e.Id)
where v.CodeConceptId in (779767,779749)
and v.Value is not NULL
and e.ClassConceptId not in (0,10067561,2649591)
and e.ClassConceptId is not NULL
and v.NormalizedValueUOMConceptId not in (0,1067561,1067557) ;"""

sbp = population.load_sql_table(sql = sql)

In [3]:
concept = pd.read_parquet('data/Concept.parquet')

In [16]:
bp.groupby('ClassConceptId').Id.nunique()

,Id,Id
ClassConceptId,,
1065216,17348,13395
1065217,2504,885
1065219,43,43
1065220,12506,456
1065225,1104,71
1065227,13,1
1065230,1,1
1067555,169,158


In [9]:
sql = """select v.CodeConceptId, e.ClassConceptId as 'EncounterTypeId', 
count(distinct v.Id) as NRecords, 
SUM(CASE WHEN v.Value is NULL THEN 1 ELSE 0 END) 
AS "NullValue", count(Value) as "NonNullValue"
from
Vital v join Encounter e on (v.EncounterId = e.Id)
where v.CodeConceptId in (779767,779749,825889)
group by v.CodeConceptId, e.ClassConceptId;"""

encounter_bp  = population.load_sql_table(sql = sql)

In [17]:
encounter_bp_with_concept = encounter_bp.merge(concept, left_on = 'CodeConceptId',right_on = 'ConceptId')

encounter_bp_with_concept = encounter_bp_with_concept.merge(concept, left_on = 'EncounterTypeId',right_on = 'ConceptId')
encounter_bp_with_concept.rename({'ConceptName_x':'VitalSignName','ConceptName_y':'EncounterName'},axis=1, inplace=True)



,CodeConceptId,VitalSignName,EncounterTypeId,EncounterName,NRecords,NullValue,NonNullValue
0,779749,Diastolic blood pressure,1065225,observation encounter,830041,1,830040
1,779767,Systolic blood pressure,1065225,observation encounter,1478539,2839,1475700
2,825889,Blood pressure systolic and diastolic,1065225,observation encounter,2040219,6722,2033497
3,779749,Diastolic blood pressure,1065230,virtual,8926,81,8845
4,779767,Systolic blood pressure,1065230,virtual,49509,285,49224
5,825889,Blood pressure systolic and diastolic,1065230,virtual,58861,7438,51423
6,779749,Diastolic blood pressure,1067555,Invalid,4,0,4
7,779767,Systolic blood pressure,1067555,Invalid,183,0,183
8,825889,Blood pressure systolic and diastolic,1067555,Invalid,360,21,339
9,779749,Diastolic blood pressure,1065217,emergency,761359,13,761346


In [21]:
enct_bp_name = encounter_bp_with_concept[['CodeConceptId','VitalSignName', 'EncounterTypeId','EncounterName','NRecords','NullValue',"NonNullValue"]]
enct_bp_name.sort_values(['VitalSignName'])

,CodeConceptId,VitalSignName,EncounterTypeId,EncounterName,NRecords,NullValue,NonNullValue
17,825889,Blood pressure systolic and diastolic,1065227,short stay,1159127,3177,1155950
32,825889,Blood pressure systolic and diastolic,1065220,inpatient encounter,33046626,108241,32938385
29,825889,Blood pressure systolic and diastolic,2649591,Insufficient Information,714071,1793,712278
26,825889,Blood pressure systolic and diastolic,1067561,No Information,52377,261,52116
23,825889,Blood pressure systolic and diastolic,1065216,ambulatory,20864390,37941,20826449
20,825889,Blood pressure systolic and diastolic,1065226,pre-admission,1745,8,1737
34,825889,Blood pressure systolic and diastolic,1065219,home health,1276636,10273,1266363
14,825889,Blood pressure systolic and diastolic,0,Field has not been mapped,734952,2154,732798
11,825889,Blood pressure systolic and diastolic,1065217,emergency,4571005,11497,4559508
35,825889,Blood pressure systolic and diastolic,1065215,inpatient acute,4425,15,4410


In [44]:
sql = """select CodeConceptId, StatusConceptId , 
count(distinct Id) as NRecords, 
SUM(CASE WHEN Value is NULL THEN 1 ELSE 0 END) 
AS "NullValue", count(Value) as "NonNullValue"
from
Vital 
where CodeConceptId in (779767,779749,825889)
group by CodeConceptId, StatusConceptId;"""

status_bp  = population.load_sql_table(sql = sql)

status_bp_with_concept = status_bp.merge(concept, left_on = 'CodeConceptId',right_on = 'ConceptId')

status_bp_with_concept = status_bp_with_concept.merge(concept, left_on = 'StatusConceptId',right_on = 'ConceptId')
status_bp_with_concept.rename({'ConceptName_x':'VitalSignName','ConceptName_y':'StatusName'},axis=1, inplace=True)
status_bp_name = status_bp_with_concept[['VitalSignName','StatusName','NRecords','NullValue',"NonNullValue"]]
status_bp_name.sort_values(['VitalSignName'])

,VitalSignName,StatusName,NRecords,NullValue,NonNullValue
18,Blood pressure systolic and diastolic,Insufficient Information,204,0,204
13,Blood pressure systolic and diastolic,Final,6018441,16378,6002063
10,Blood pressure systolic and diastolic,Field is not present in source,85720378,258187,85462191
7,Blood pressure systolic and diastolic,Field has not been mapped,2,0,2
15,Diastolic blood pressure,No Information,119,0,119
12,Diastolic blood pressure,Final,9237024,0,9237024
17,Diastolic blood pressure,Insufficient Information,826408,69,826339
9,Diastolic blood pressure,Field is not present in source,629727,592,629135
4,Diastolic blood pressure,Amended,22114,0,22114
2,Diastolic blood pressure,Incomplete,606,0,606
